In [1]:
! pip install googledrivedownloader

In [2]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1jlzvS4GF56vfxb5Wm_4-TrUgICz4KYDs',
                                    dest_path='./data/news.zip',
                                    unzip=True)

Unzipping...Done.


In [1]:
import re
import pandas as pd
import numpy as np
from gensim.models.fasttext import FastText

In [2]:
def clean_text(text):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [3]:
df = pd.read_csv('../../../Data/category-data/news-files/india-news-headlines.csv')
df = df.drop(['publish_date','headline_category'], axis=1)
df.columns = ['text']
df['text'] = df['text'].apply(lambda x: clean_text(x.lower()))
df.head()

,text
0,win over cena satisfying but defeating underta...
1,raju chacha
2,status quo will not be disturbed at ayodhya sa...
3,fissures in hurriyat over pak visit
4,america unwanted heading for india


In [8]:
sentences = df.text.to_list()
sentences[2019]

'sail quarters are on sale'

In [0]:
with open('train.txt', 'w') as fout:
  for sent in sentences:
    fout.write("%s\n" %sent)

In [10]:
! git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 3488 (delta 0), reused 2 (delta 0), pack-reused 3483
Receiving objects: 100% (3488/3488), 8.01 MiB | 5.70 MiB/s, done.
Resolving deltas: 100% (2193/2193), done.


In [11]:
%cd fastText
! make

/content/fastText
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/quantmatrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/vector.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/model.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/utils.cc
c++ -pthread -std=c++11 -march=native -O3 -funrol

In [12]:
%cd ..
! mkdir model
! ls -l
%cd fastText

/content
total 130404
drwxr-xr-x  2 root root      4096 Sep  8 11:55 data
drwxr-xr-x 12 root root      4096 Sep  8 11:57 fastText
drwxr-xr-x  2 root root      4096 Sep  8 11:57 model
drwxr-xr-x  1 root root      4096 Aug 27 16:17 sample_data
-rw-r--r--  1 root root 133513401 Sep  8 11:57 train.txt
/content/fastText


In [13]:
! ./fasttext skipgram -input ../train.txt -output ../model/fasttext-model -epoch 15

Read 25M words
Number of words:  70654
Number of labels: 0
Progress: 100.0% words/sec/thread:   15155 lr:  0.000000 avg.loss:  1.447002 ETA:   0h 0m 0s


In [14]:
import keras
from keras.preprocessing import text, sequence
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Input, Dropout, Embedding, LSTM, SpatialDropout1D, Flatten
from keras.layers import GRU, Bidirectional, Convolution1D, GlobalMaxPool1D, TimeDistributed, MaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import add

from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import time
from sklearn.model_selection import train_test_split as tts
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [0]:
from gensim.models.wrappers import FastText
model = FastText.load_fasttext_format('../model/fasttext-model.bin')

In [0]:
def clean_text(text):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    if len(text.split()) < 8:
      return np.nan
    return text

In [16]:
gdd.download_file_from_google_drive(file_id='1t2bs-j0ZWmh6igA9miD4d8WJaUOCF7nI',
                                    dest_path='./dummy.csv')

In [17]:
df = pd.read_csv('dummy.csv')
print('df Shape:', df.shape)
df = df.where(df['video']==0).dropna()
df = df.drop('video', axis=1)
df['id'] = df['id'].astype('int64')
for col in df.columns[2:]:
    df[col] = df[col].astype('int8')
print('Dataset shape:', df.shape)
df['headline'] = df['headline'].apply(lambda text: clean_text(text))
df = df.dropna()
print('Length of df:', len(df))

labels = np.argmax(df.iloc[:, 2:].values, axis=1).tolist()
print('Length of labels:', len(labels))

df['labels'] = labels
df = df.iloc[:, [1,16]]

df Shape: (638842, 17)
Dataset shape: (636494, 16)
Length of df: 636494
Length of labels: 636494


In [18]:
X = df['headline']
y = pd.get_dummies(df.labels)

train_x, valid_x, train_y, valid_y = tts(X, y, test_size=0.2, random_state=0)
print('train_x shape:', train_x.shape)
print('valid_x shape:', valid_x.shape)
print('train_y shape:', train_y.shape)
print('valid_y shape:', valid_y.shape)

train_x shape: (509195,)
valid_x shape: (127299,)
train_y shape: (509195, 14)
valid_y shape: (127299, 14)


In [0]:
token = text.Tokenizer()
token.fit_on_texts(X)
word_index = token.word_index

In [0]:
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=60)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=60)

In [21]:
embedding_matrix = np.zeros((len(word_index)+1, 100))
ct = 0
for word, i in word_index.items():
  try:
    embedding_matrix[i] = model[word]
  except:
    embedding_matrix[i] = [0.0]*100
    ct+=1
      
print('{} out of {} words absent'.format(ct, i))

59 out of 99693 words absent


In [0]:
def plot_neural_net_details(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def train_model(classifier, train, label, valid, model_name, epochs=2):
    start = time.time()

    early_stopping = EarlyStopping(monitor='val_loss', patience=2, min_delta=0.0001)
    model_checkpoint = ModelCheckpoint(model_name+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)        
    
    history = classifier.fit(train, label, epochs=epochs, batch_size=32, validation_split=0.1,
                             callbacks=[early_stopping, model_checkpoint])
    
    predictions = classifier.predict(valid)
    print("Time taken: %.2f seconds" % (time.time()-start))
    plot_neural_net_details(history)
    return predictions, classifier

### CNN

In [0]:
def create_cnn():
    # Add an Input Layer
    input_layer = Input((60, ))

    embedding_layer = Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = SpatialDropout1D(0.3)(embedding_layer)

    conv_layer1 = Convolution1D(100, 3, activation="relu")(embedding_layer)    
    pooling_layer1 = GlobalMaxPool1D()(conv_layer1)
    
    output_layer1 = Dense(50, activation='relu')(pooling_layer1)
    output_layer1 = Dropout(0.25)(output_layer1)
    
    output_layer2 = Dense(14, activation='softmax')(output_layer1)

    model = Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

classifier = create_cnn()
cnn_pred, cnn = train_model(classifier, train_seq_x, train_y, valid_seq_x, 'CNN', epochs=10)

In [0]:
print('CNN accuracy on test: %.3f' % (accuracy_score(np.argmax(cnn_pred, axis=1), valid_y.idxmax(axis=1))))

### Bi-LSTM

In [0]:
def create_bi_lstm():
    input_layer = Input((60, ))

    embedding_layer = Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = SpatialDropout1D(0.3)(embedding_layer)

    lstm_layer = Bidirectional(LSTM(100))(embedding_layer)

    output_layer1 = Dense(50, activation='relu')(lstm_layer)
    output_layer1 = Dropout(0.25)(output_layer1)
    output_layer2 = Dense(14, activation='softmax')(output_layer1)

    model = Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

classifier = create_bi_lstm()
bilstm_pred, bilstm = train_model(classifier, train_seq_x, train_y, valid_seq_x, 'Bi-LSTM', 10)

In [0]:
print('Bi-LSTM accuracy on test: %.3f' % (accuracy_score(np.argmax(bilstm_pred, axis=1), valid_y.idxmax(axis=1))))

### R-CNN

In [0]:
def create_rcnn():
    input_layer = Input((60, ))
    
    embedding_layer = Embedding(len(word_index) + 1, 100, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = SpatialDropout1D(0.3)(embedding_layer)
        
    rnn_layer = Bidirectional(GRU(50, return_sequences=True))(embedding_layer)
    
    conv_layer = Convolution1D(100, 3, activation="relu")(rnn_layer)

    pooling_layer = GlobalMaxPool1D()(conv_layer)

    output_layer1 = Dense(50, activation='relu')(pooling_layer)
    output_layer1 = Dropout(0.25)(output_layer1)
    output_layer2 = Dense(14, activation='softmax')(output_layer1)

    model = Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

classifier = create_rcnn()
rcnn_pred, rcnn = train_model(classifier, train_seq_x, train_y, valid_seq_x, 'R-CNN', epochs=10)

In [0]:
print('R-CNN accuracy on test: %.3f' % (accuracy_score(np.argmax(rcnn_pred, axis=1), valid_y.idxmax(axis=1))))